# DRAFT mini Project

# Boss's Orders

Task:

Our company is not working efficiently! We spend way too much time
drafting documents and this needs to be fixed!

For the company, you need to create an Al Agentic System that can
speed up drafting documents, emails, etc. The Al Agentic System
should have Human-Al Collaboration meaning the Human should be
able to able to provide continuous feedback and the Al Agent shoul
stop when the Human is happy with the draft. The system should a
be fast and be able to save the drafts


In [1]:
from typing import Annotated, Sequence, TypedDict
from dotenv import load_dotenv  
from langchain_core.messages import BaseMessage, HumanMessage, AIMessage, ToolMessage, SystemMessage
from langchain_openai import ChatOpenAI
from langchain_core.tools import tool
from langgraph.graph.message import add_messages
from langgraph.graph import StateGraph, END
from langgraph.prebuilt import ToolNode


In [2]:
load_dotenv()

True

In [3]:
# This is the global variable to store document content
document_content = ""

In [4]:
class AgentState(TypedDict):
    messages: Annotated[Sequence[BaseMessage], add_messages]

In [5]:
@tool
def update(content: str) -> str:
    """Updates the document with the provided content."""
    global document_content
    document_content = content
    return f"Document has been updated successfully! The current content is:\n{document_content}"


@tool
def save(filename: str) -> str:
    """Save the current document to a text file and finish the process.
    
    Args:
        filename: Name for the text file.
    """

    global document_content

    if not filename.endswith('.txt'):
        filename = f"{filename}.txt"


    try:
        with open(filename, 'w') as file:
            file.write(document_content)
        print(f"\n💾 Document has been saved to: {filename}")
        return f"Document has been saved successfully to '{filename}'."
    
    except Exception as e:
        return f"Error saving document: {str(e)}"

In [6]:
tools = [update, save]

model = ChatOpenAI(model="gpt-4o").bind_tools(tools)

# Draft Agent

In [8]:
def our_agent(state: AgentState) -> AgentState:
    system_prompt = SystemMessage(content=f"""
    You are Drafter, a helpful writing assistant. You are going to help the user update and modify documents.
    
    - If the user wants to update or modify content, use the 'update' tool with the complete updated content.
    - If the user wants to save and finish, you need to use the 'save' tool.
    - Make sure to always show the current document state after modifications.
    
    The current document content is:{document_content}
    """)

    if not state["messages"]:
        user_input = "I'm ready to help you update a document. What would you like to create?"
        user_message = HumanMessage(content=user_input)

    else:
        user_input = input("\nWhat would you like to do with the document? ")
        print(f"\n👤 USER: {user_input}")
        user_message = HumanMessage(content=user_input)

    all_messages = [system_prompt] + list(state["messages"]) + [user_message]

    response = model.invoke(all_messages)

    print(f"\n🤖 AI: {response.content}")
    if hasattr(response, "tool_calls") and response.tool_calls:
        print(f"🔧 USING TOOLS: {[tc['name'] for tc in response.tool_calls]}")

    return {"messages": list(state["messages"]) + [user_message, response]}

In [9]:
def should_continue(state: AgentState) -> str:
    """Determine if we should continue or end the conversation."""

    messages = state["messages"]
    
    if not messages:
        return "continue"
    
    # This looks for the most recent tool message....
    for message in reversed(messages):
        # ... and checks if this is a ToolMessage resulting from save
        if (isinstance(message, ToolMessage) and 
            "saved" in message.content.lower() and
            "document" in message.content.lower()):
            return "end" # goes to the end edge which leads to the endpoint
        
    return "continue"

In [11]:
def print_messages(messages):
    """Function I made to print the messages in a more readable format"""
    if not messages:
        return
    
    for message in messages[-3:]:
        if isinstance(message, ToolMessage):
            print(f"\n🛠️ TOOL RESULT: {message.content}")

In [12]:
graph = StateGraph(AgentState)

graph.add_node("agent", our_agent)
graph.add_node("tools", ToolNode(tools))

graph.set_entry_point("agent")

graph.add_edge("agent", "tools")


graph.add_conditional_edges(
    "tools",
    should_continue,
    {
        "continue": "agent",
        "end": END,
    },
)

In [13]:
app = graph.compile()

In [14]:
def run_document_agent():
    print("\n ===== DRAFTER =====")
    
    state = {"messages": []}
    
    for step in app.stream(state, stream_mode="values"):
        if "messages" in step:
            print_messages(step["messages"])
    
    print("\n ===== DRAFTER FINISHED =====")

if __name__ == "__main__":
    run_document_agent()



 ===== DRAFTER =====

🤖 AI: We don't have any content stored in the document yet. Please provide some text or let me know what you'd like to create or include, and I'll help you with it.



What would you like to do with the document?  generate a sick leave mail



👤 USER: generate a sick leave mail

🤖 AI: 
🔧 USING TOOLS: ['update']

🛠️ TOOL RESULT: Document has been updated successfully! The current content is:
Subject: Sick Leave Request

Dear [Manager's Name],

I hope this message finds you well. I am writing to inform you that I am unable to attend work due to a sudden illness. Regrettably, I will need to take sick leave starting today, [start date], and will keep you updated on my health progress.

I plan to visit my doctor and will follow their recommendations regarding the duration of my leave. I will ensure that all my pending tasks are up to date and will hand over any important responsibilities to [Colleague's Name] during my absence.

I apologize for any inconvenience my absence might cause and thank you for your understanding. Please let me know if there are any forms or formalities I need to complete from my side.

Thank you for your consideration.

Warm regards,

[Your Name]



What would you like to do with the document?  Manager name is sai



👤 USER: Manager name is sai

🤖 AI: 
🔧 USING TOOLS: ['update']

🛠️ TOOL RESULT: Document has been updated successfully! The current content is:
Subject: Sick Leave Request

Dear [Manager's Name],

I hope this message finds you well. I am writing to inform you that I am unable to attend work due to a sudden illness. Regrettably, I will need to take sick leave starting today, [start date], and will keep you updated on my health progress.

I plan to visit my doctor and will follow their recommendations regarding the duration of my leave. I will ensure that all my pending tasks are up to date and will hand over any important responsibilities to [Colleague's Name] during my absence.

I apologize for any inconvenience my absence might cause and thank you for your understanding. Please let me know if there are any forms or formalities I need to complete from my side.

Thank you for your consideration.

Warm regards,

[Your Name]

🛠️ TOOL RESULT: Document has been updated successfully! The cur


What would you like to do with the document?  extending the leave tomorrow as well



👤 USER: extending the leave tomorrow as well

🤖 AI: 
🔧 USING TOOLS: ['update']

🛠️ TOOL RESULT: Document has been updated successfully! The current content is:
Subject: Sick Leave Request

Dear Sai,

I hope this message finds you well. I am writing to inform you that I am unable to attend work due to a sudden illness. Regrettably, I will need to take sick leave starting today, [start date], and will keep you updated on my health progress.

I plan to visit my doctor and will follow their recommendations regarding the duration of my leave. I will ensure that all my pending tasks are up to date and will hand over any important responsibilities to [Colleague's Name] during my absence.

I apologize for any inconvenience my absence might cause and thank you for your understanding. Please let me know if there are any forms or formalities I need to complete from my side.

Thank you for your consideration.

Warm regards,

[Your Name]

🛠️ TOOL RESULT: Document has been updated successfully! The


What would you like to do with the document?  great save this



👤 USER: great save this

🤖 AI: 
🔧 USING TOOLS: ['save']

🛠️ TOOL RESULT: Document has been updated successfully! The current content is:
Subject: Sick Leave Request

Dear Sai,

I hope this message finds you well. I am writing to inform you that I am unable to attend work due to a sudden illness. Regrettably, I will need to take sick leave starting today and extending through tomorrow as well. I will keep you updated on my health progress.

I plan to visit my doctor and will follow their recommendations regarding the duration of my leave. I will ensure that all my pending tasks are up to date and will hand over any important responsibilities to [Colleague's Name] during my absence.

I apologize for any inconvenience my absence might cause and thank you for your understanding. Please let me know if there are any forms or formalities I need to complete from my side.

Thank you for your consideration.

Warm regards,

[Your Name]

💾 Document has been saved to: Sick_Leave_Request.txt

🛠️ TO